#!conda install -y cudatoolkit
#!pip install bitsandbytes
!pip install transformers==4.33.2
!pip install datasets
!pip install evaluate
!pip install peft
!pip install accelerate -U
!pip install transformers[torch]
!pip install -r requirements.txt

In [2]:
%pip install -q -U torch==2.0.1 bitsandbytes==0.40.2
%pip install -q -U transformers==4.31.0 peft==0.4.0 accelerate==0.21.0
%pip install -q -U datasets py7zr einops tensorboardX
!pip install evaluate
# Add installed cuda runtime to path for bitsandbytes
import os
import nvidia

cuda_install_dir = '/'.join(nvidia.__file__.split('/')[:-1]) + '/cuda_runtime/lib/'
os.environ['LD_LIBRARY_PATH'] =  cuda_install_dir


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any

In [3]:
from collections import defaultdict
import copy
import json
import os
import sys
#sys.path.append("/home/sagemaker-user/bitsandbytes")
#os.environ['LD_LIBRARY_PATH'] = '/opt/conda/lib/'
from os.path import exists, join, isdir
from dataclasses import dataclass, field
import sys
from typing import Optional, Dict, Sequence
import numpy as np
from tqdm import tqdm
import logging
#import bitsandbytes as bnb
import pandas as pd

import torch
torch.cuda.init()
print(torch.cuda.is_available())
import os
#os.environ['TRANSFORMERS_CACHE'] = '/project/SDS/research/christ_research/Llama 2/llama2-7b/cache'
import transformers
from torch.nn.utils.rnn import pad_sequence
import argparse
# from transformers import (
#     AutoTokenizer,
#     AutoModelForCausalLM,
#     set_seed,
#     Seq2SeqTrainer,
# #    BitsAndBytesConfig,
#     LlamaTokenizer

# )
from datasets import load_dataset, Dataset
import evaluate

# from peft import (
#     prepare_model_for_kbit_training,
#     LoraConfig,
#     get_peft_model,
#     PeftModel,
#     TaskType
# )
#from peft.tuners.lora import LoraLayer
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
#from dotenv import load_dotenv
import os
torch.backends.cuda.matmul.allow_tf32 = True

# Load the environmental variables from the .env file
#load_dotenv()

#token = os.getenv('text_classifier_token')

#from huggingface_hub import login
#login(token = token)

from datasets import load_dataset, DatasetDict

True


In [4]:
import random

In [95]:
df = pd.read_csv('data/appropriateness.csv')
df['label'] = df['label'].astype('int')
pos = df[df['label']==1]
neg = df[df['label']==0]
neg_len = len(neg)
pos = pos.sample(n = neg_len, replace = False)
df2 = pd.concat([pos, neg])
df2 = df2.sample(frac=1)
df2.reset_index(drop=True, inplace=True)
df2.to_csv('data/appropriateness_balanced.csv')
df2

,Unnamed: 0,label,text
0,834,1,Question: LeBron James is 6 feet 9 inches tall...
1,1329,1,Question: The volleyball team has 12 members. ...
2,742,1,Question: There are 50 cars in a parking lot. ...
3,414,1,"Question: In Super Mario, the mushroom power-u..."
4,1519,1,Question: Spiderman can spin 10 webs in 15 min...
...,...,...,...
979,1496,1,Question: Patrick Mahomes has 200 touchdowns. ...
980,906,1,Question: Barbie is a fashionista. She has 300...
981,403,0,Question: A cheerleading squad of 20 members i...
982,1063,0,Question: The 6 Power Rangers each have a Mega...


In [96]:
# Load the dataset
dataset = load_dataset('csv', data_files="data/appropriateness_balanced.csv")
#dataset = load_dataset('csv', data_files="balanced.csv")

# Get the total number of examples in the dataset
# total_examples = len(dataset['train'])

# # Calculate the sizes of the training, test, and validation sets
# train_size = int(0.8 * total_examples)
# test_size = int(0.1 * total_examples)
# valid_size = total_examples - train_size - test_size

# # Manually split the dataset into training, test, and validation sets
# train_dataset = dataset['train'].shuffle(seed=42).select(range(train_size))
# test_dataset = dataset['train'].shuffle(seed=42).select(range(train_size, train_size + test_size))
# valid_dataset = dataset['train'].shuffle(seed=42).select(range(train_size + test_size, total_examples))

dataset_train_valid_test = dataset['train'].train_test_split(test_size = .2, seed = 42)
dataset_valid_test = dataset_train_valid_test['test'].train_test_split(test_size = .5, seed = 42)
# Create a DatasetDict to hold the splits
train_test_valid_dataset = DatasetDict({
    'train': dataset_train_valid_test['train'],
    'test': dataset_valid_test['train'],
    'valid': dataset_valid_test['test']
})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [97]:
train_test_valid_dataset['valid'][2]

{'Unnamed: 0.1': 922,
 'Unnamed: 0': 422,
 'label': 1,
 'text': 'Question: Captain Marvel has 3000 vibranium-powered punches. She uses 200 punches in the morning, 400 punches in the afternoon, and 100 punches in the evening. How many vibranium-powered punches does she have left?\nSolution:\ndef solution():\n    #Captain Marvel started with 3000 punches\n    punches_initial = 3000\n    #She used 200 in the morning\n    punches_morning = 200\n    #400 in the afternoon\n    punches_afternoon = 400\n    #100 in the evening\n    punches_evening = 100\n    #The answer is\n    result = punches_initial - punches_morning - punches_afternoon - punches_evening\n    return result\nAnswer: 2300'}

In [98]:
#Set up tokenizer
from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", use_auth_token=True)
# tokenizer.add_special_tokens({"pad_token":"[PAD]"})

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
#tokenizer.add_special_tokens({"pad_token":"[PAD]"})

#Preprocess and collate data
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train_test_valid_dataset = train_test_valid_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/787 [00:00<?, ? examples/s]

Map:   0%|          | 0/98 [00:00<?, ? examples/s]

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

In [99]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#Prepare evaluation function
import evaluate
accuracy = evaluate.load("accuracy")

import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# roc_auc = evaluate.load("roc_auc")

# import numpy as np
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     return roc_auc.compute(prediction_scores=predictions, references=labels)

#Training
id2label = {0: "NOT APPROPRIATE", 1: "APPROPRIATE"}
label2id = {"NOT APPROPRIATE": 0, "APPROPRIATE": 1}

In [100]:
#Import model
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# model = AutoModelForSequenceClassification.from_pretrained(
#     "meta-llama/Llama-2-7b-hf", num_labels=2, id2label=id2label, label2id=label2id,
#         use_auth_token=True,  
#       # max_memory=max_memory,
#       # torch_dtype=torch.bfloat16, 
#       device_map = 'auto',
#         quantization_config=BitsAndBytesConfig(
#             load_in_4bit=True,
#             bnb_4bit_compute_dtype=torch.bfloat16,
#             bnb_4bit_use_double_quant=True,
#             bnb_4bit_quant_type='nf4'))

# model = AutoModelForSequenceClassification.from_pretrained(
#     "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id,
#         use_auth_token=True,  
#       # max_memory=max_memory,
#       # torch_dtype=torch.bfloat16, 
#         quantization_config=BitsAndBytesConfig(
#             load_in_4bit=True,
#             bnb_4bit_compute_dtype=torch.bfloat16,
#             bnb_4bit_use_double_quant=True,
#             bnb_4bit_quant_type='nf4'))

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [102]:
#NEWLY ADDED
# model.gradient_checkpointing_enable()
# model = prepare_model_for_kbit_training(model)

# peft_config = LoraConfig(
#     task_type=TaskType.SEQ_CLS,
#     r=16,
#     lora_alpha=32,
#     lora_dropout=0.1,
# )      

# model = get_peft_model(model, peft_config)

#END NEWLY ADDED
# training_args = TrainingArguments(
#     output_dir="text_classifier_llama",
#     learning_rate=2e-5,
#     per_device_train_batch_size=1,
#   # auto_find_batch_size = True,
#     num_train_epochs=8,
#     weight_decay=0.01,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True, 
#     bf16=True)

training_args = TrainingArguments(
    output_dir="appropriateness_classifier",
    learning_rate=.0001,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=16,
    weight_decay=0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_test_valid_dataset["train"],
    eval_dataset=tokenized_train_test_valid_dataset["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

#trainer.train(resume_from_checkpoint=True)
trainer.train()
print(tokenized_train_test_valid_dataset["test"])
test_input_data = tokenized_train_test_valid_dataset["test"]["input_ids"]

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.321921,0.888889
2,No log,0.472435,0.858586
3,No log,0.437931,0.858586
4,No log,0.463053,0.858586
5,No log,0.482812,0.838384
6,No log,0.518507,0.848485
7,No log,0.646680,0.818182
8,No log,0.524509,0.848485
9,No log,0.549167,0.848485
10,0.260500,0.618347,0.838384


Dataset({
    features: ['Unnamed: 0.1', 'Unnamed: 0', 'label', 'text', 'input_ids', 'attention_mask'],
    num_rows: 98
})


In [103]:
# predictions = trainer.predict(test_dataset=test_input_data)
# print(predictions)
trainer.evaluate(eval_dataset = tokenized_train_test_valid_dataset["test"])

{'eval_loss': 0.4093845784664154,
 'eval_accuracy': 0.8469387755102041,
 'eval_runtime': 0.5556,
 'eval_samples_per_second': 176.378,
 'eval_steps_per_second': 12.598,
 'epoch': 16.0}

In [104]:
trainer.predict(test_dataset = tokenized_train_test_valid_dataset["test"])

PredictionOutput(predictions=array([[-0.6207591 ,  0.6736535 ],
       [ 1.0703156 , -1.0787371 ],
       [ 1.0732976 , -1.083002  ],
       [ 1.0771855 , -1.0870489 ],
       [ 1.0770115 , -1.0839195 ],
       [ 1.0791233 , -1.0896196 ],
       [-0.8348101 ,  0.954373  ],
       [-0.7923826 ,  0.8939277 ],
       [-0.4819457 ,  0.4984986 ],
       [-0.8402348 ,  0.957724  ],
       [ 1.0282779 , -1.0092639 ],
       [ 0.97511965, -0.9188721 ],
       [ 0.94950694, -0.8858211 ],
       [ 1.0813584 , -1.0934119 ],
       [-0.91332245,  1.073796  ],
       [-0.8381578 ,  0.94643205],
       [ 0.95713544, -0.907661  ],
       [-0.8014436 ,  0.8927865 ],
       [ 1.06014   , -1.0623937 ],
       [-0.76129806,  0.8426286 ],
       [-0.690245  ,  0.754299  ],
       [ 0.75947154, -0.7635017 ],
       [ 0.98622674, -0.93695325],
       [-0.8832992 ,  1.0191603 ],
       [ 1.0769575 , -1.0843459 ],
       [ 0.99102783, -0.9393181 ],
       [-0.78705055,  0.88623387],
       [-0.51369905,  0.53

## Solvability Classifier

In [105]:
df = pd.read_csv('data/solvability.csv')
df['label'] = df['label'].astype('int')
pos = df[df['label']==1]
neg = df[df['label']==0]
neg_len = len(neg)
pos = pos.sample(n = neg_len, replace = False)
df2 = pd.concat([pos, neg])
df2 = df2.sample(frac=1)
df2.reset_index(drop=True, inplace=True)
df2.to_csv('data/solvability_balanced.csv')
df2

,Unnamed: 0,label,text
0,603,1,Question: The lacrosse team has 54 sticks. The...
1,1440,0,Question: Spiderman has 20 spiderwebs. He uses...
2,1201,1,Question: Patrick Mahomes has 2000 passing yar...
3,1594,1,Question: A field hockey game has 11 players o...
4,765,0,Question: Iron Man has 152 armor suits. He has...
...,...,...,...
447,300,0,Question: The 2:00 PM train from Penn Station ...
448,1634,1,Question: 2000 people were at a rally. 1000 of...
449,71,0,"Question: In the Mushroom Kingdom, there are 1..."
450,117,0,Question: 5 cats and 4 dogs are for sale at an...


In [110]:
# Load the dataset
dataset = load_dataset('csv', data_files="data/solvability_balanced.csv")
#dataset = load_dataset('csv', data_files="balanced.csv")

# Get the total number of examples in the dataset
# total_examples = len(dataset['train'])

# # Calculate the sizes of the training, test, and validation sets
# train_size = int(0.8 * total_examples)
# test_size = int(0.1 * total_examples)
# valid_size = total_examples - train_size - test_size

# # Manually split the dataset into training, test, and validation sets
# train_dataset = dataset['train'].shuffle(seed=42).select(range(train_size))
# test_dataset = dataset['train'].shuffle(seed=42).select(range(train_size, train_size + test_size))
# valid_dataset = dataset['train'].shuffle(seed=42).select(range(train_size + test_size, total_examples))

dataset_train_valid_test = dataset['train'].train_test_split(test_size = .2, seed = 42)
dataset_valid_test = dataset_train_valid_test['test'].train_test_split(test_size = .5, seed = 42)
# Create a DatasetDict to hold the splits
train_test_valid_dataset = DatasetDict({
    'train': dataset_train_valid_test['train'],
    'test': dataset_valid_test['train'],
    'valid': dataset_valid_test['test']
})

In [111]:
#Set up tokenizer
from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", use_auth_token=True)
# tokenizer.add_special_tokens({"pad_token":"[PAD]"})

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
#tokenizer.add_special_tokens({"pad_token":"[PAD]"})

#Preprocess and collate data
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train_test_valid_dataset = train_test_valid_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

In [112]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#Prepare evaluation function
import evaluate
accuracy = evaluate.load("accuracy")

import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# roc_auc = evaluate.load("roc_auc")

# import numpy as np
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     return roc_auc.compute(prediction_scores=predictions, references=labels)

#Training
id2label = {0: "NOT SOLVABLE", 1: "SOLVABLE"}
label2id = {"NOT SOLVABLE": 0, "SOLVABLE": 1}

In [113]:
#Import model
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# model = AutoModelForSequenceClassification.from_pretrained(
#     "meta-llama/Llama-2-7b-hf", num_labels=2, id2label=id2label, label2id=label2id,
#         use_auth_token=True,  
#       # max_memory=max_memory,
#       # torch_dtype=torch.bfloat16, 
#       device_map = 'auto',
#         quantization_config=BitsAndBytesConfig(
#             load_in_4bit=True,
#             bnb_4bit_compute_dtype=torch.bfloat16,
#             bnb_4bit_use_double_quant=True,
#             bnb_4bit_quant_type='nf4'))

# model = AutoModelForSequenceClassification.from_pretrained(
#     "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id,
#         use_auth_token=True,  
#       # max_memory=max_memory,
#       # torch_dtype=torch.bfloat16, 
#         quantization_config=BitsAndBytesConfig(
#             load_in_4bit=True,
#             bnb_4bit_compute_dtype=torch.bfloat16,
#             bnb_4bit_use_double_quant=True,
#             bnb_4bit_quant_type='nf4'))

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [114]:
training_args = TrainingArguments(
    output_dir="solvability_classifier",
    learning_rate=.0001,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=8,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_test_valid_dataset["train"],
    eval_dataset=tokenized_train_test_valid_dataset["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
for param in model.bert.parameters():
    param.requires_grad = False
#trainer.train(resume_from_checkpoint=True)
trainer.train()
print(tokenized_train_test_valid_dataset["test"])
test_input_data = tokenized_train_test_valid_dataset["test"]["input_ids"]

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.710070,0.391304
2,No log,0.745079,0.369565
3,No log,0.709998,0.391304
4,No log,0.708976,0.391304
5,No log,0.702449,0.369565
6,No log,0.722045,0.369565


KeyboardInterrupt: 

In [39]:
# predictions = trainer.predict(test_dataset=test_input_data)
# print(predictions)
trainer.evaluate(eval_dataset = tokenized_train_test_valid_dataset["test"])

{'eval_loss': 0.4754084646701813,
 'eval_accuracy': 0.8242424242424242,
 'eval_runtime': 1.6531,
 'eval_samples_per_second': 99.81,
 'eval_steps_per_second': 6.654,
 'epoch': 8.0}

In [40]:
trainer.predict(test_dataset = tokenized_train_test_valid_dataset["test"])

PredictionOutput(predictions=array([[-1.1767561 ,  0.85649323],
       [-1.0614527 ,  0.8524137 ],
       [-1.1524765 ,  0.9674901 ],
       [-1.169087  ,  0.85458505],
       [-1.118228  ,  0.9000377 ],
       [-1.0491871 ,  0.88732153],
       [-1.0239848 ,  0.929475  ],
       [-1.1332685 ,  0.91266596],
       [-1.0671339 ,  0.9459554 ],
       [-1.1351367 ,  0.8960755 ],
       [-1.1527706 ,  0.86005044],
       [-1.0023655 ,  0.8505782 ],
       [-1.1405097 ,  0.8213093 ],
       [-1.0786202 ,  0.8799991 ],
       [-1.1250837 ,  0.88174665],
       [-1.02384   ,  0.8139355 ],
       [-1.1747395 ,  0.82902414],
       [-1.1255609 ,  0.92080224],
       [-0.90249366,  1.0001096 ],
       [-1.0512965 ,  0.8697106 ],
       [-1.0555145 ,  0.945542  ],
       [-1.0722532 ,  0.88484186],
       [-1.1100919 ,  0.9010052 ],
       [-1.0702143 ,  0.9194692 ],
       [-1.1220598 ,  0.8647202 ],
       [-1.088548  ,  0.9485705 ],
       [-1.1189343 ,  0.8919009 ],
       [-1.2064078 ,  0.79

## Accuracy Classifier

In [83]:
df = pd.read_csv('data/accuracy.csv')
df['label'] = df['label'].astype('int')
pos = df[df['label']==1]
neg = df[df['label']==0]
neg_len = len(neg)
pos = pos.sample(n = neg_len, replace = False)
df2 = pd.concat([pos, neg])
df2 = df2.sample(frac=1)
df2.reset_index(drop=True, inplace=True)
df2.to_csv('data/accuracy_balanced.csv')
df2

,Unnamed: 0.1,Unnamed: 0,label,text
0,1408,1593,1,Question: A 250-car freight train is 3.5 miles...
1,1449,1638,1,Question: Batman has 32 batarangs. He buys 4 m...
2,779,879,0,Question: The 2:00 PM train from Penn Station ...
3,570,640,0,Question: A 15-pound cake is to be cut into 30...
4,1149,1291,1,Question: Thor has 2000 thunderbolts. He uses ...
...,...,...,...,...
357,841,948,0,Question: Captain America is in a rush to save...
358,1027,1164,0,Question: 100 doggos are barking at the moon. ...
359,251,281,1,Question: 1200 people were at the 1996 Olympic...
360,1293,1454,0,Question: A bakery made 300 muffins. 150 of th...


In [84]:
# Load the dataset
dataset = load_dataset('csv', data_files="data/accuracy.csv")
#dataset = load_dataset('csv', data_files="balanced.csv")

# Get the total number of examples in the dataset
# total_examples = len(dataset['train'])

# # Calculate the sizes of the training, test, and validation sets
# train_size = int(0.8 * total_examples)
# test_size = int(0.1 * total_examples)
# valid_size = total_examples - train_size - test_size

# # Manually split the dataset into training, test, and validation sets
# train_dataset = dataset['train'].shuffle(seed=42).select(range(train_size))
# test_dataset = dataset['train'].shuffle(seed=42).select(range(train_size, train_size + test_size))
# valid_dataset = dataset['train'].shuffle(seed=42).select(range(train_size + test_size, total_examples))

dataset_train_valid_test = dataset['train'].train_test_split(test_size = .2, seed = 42)
dataset_valid_test = dataset_train_valid_test['test'].train_test_split(test_size = .5, seed = 42)
# Create a DatasetDict to hold the splits
train_test_valid_dataset = DatasetDict({
    'train': dataset_train_valid_test['train'],
    'test': dataset_valid_test['train'],
    'valid': dataset_valid_test['test']
})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [85]:
#Set up tokenizer
from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", use_auth_token=True)
# tokenizer.add_special_tokens({"pad_token":"[PAD]"})

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
#tokenizer.add_special_tokens({"pad_token":"[PAD]"})

#Preprocess and collate data
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train_test_valid_dataset = train_test_valid_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/289 [00:00<?, ? examples/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Map:   0%|          | 0/37 [00:00<?, ? examples/s]

In [86]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#Prepare evaluation function
import evaluate
accuracy = evaluate.load("accuracy")

import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# roc_auc = evaluate.load("roc_auc")

# import numpy as np
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     return roc_auc.compute(prediction_scores=predictions, references=labels)

#Training
id2label = {0: "NOT ACCURATE", 1: "ACCURATE"}
label2id = {"NOT ACCURATE": 0, "ACCURATE": 1}

In [87]:
#Import model
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# model = AutoModelForSequenceClassification.from_pretrained(
#     "meta-llama/Llama-2-7b-hf", num_labels=2, id2label=id2label, label2id=label2id,
#         use_auth_token=True,  
#       # max_memory=max_memory,
#       # torch_dtype=torch.bfloat16, 
#       device_map = 'auto',
#         quantization_config=BitsAndBytesConfig(
#             load_in_4bit=True,
#             bnb_4bit_compute_dtype=torch.bfloat16,
#             bnb_4bit_use_double_quant=True,
#             bnb_4bit_quant_type='nf4'))

# model = AutoModelForSequenceClassification.from_pretrained(
#     "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id,
#         use_auth_token=True,  
#       # max_memory=max_memory,
#       # torch_dtype=torch.bfloat16, 
#         quantization_config=BitsAndBytesConfig(
#             load_in_4bit=True,
#             bnb_4bit_compute_dtype=torch.bfloat16,
#             bnb_4bit_use_double_quant=True,
#             bnb_4bit_quant_type='nf4'))

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [88]:
import torch.nn as nn
num_ftrs = model.classifier.in_features
num_classes = 2
model.classifier = nn.Sequential(
    nn.Linear(num_ftrs, 2048),
    nn.LeakyReLU(negative_slope=0.01, inplace=True),         # Activation function (you can choose other activation functions too)
    nn.Dropout(0.4),               # Dropout layer with 20% probability
    nn.Linear(2048, num_classes)    # Final prediction fc layer
)

for param in model.bert.parameters():
    param.requires_grad = False
    
for param in model.classifier.parameters():
    param.requires_grad = True

In [92]:
training_args = TrainingArguments(
    output_dir="accuracy_classifier",
    learning_rate=.0001,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=16,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_test_valid_dataset["train"],
    eval_dataset=tokenized_train_test_valid_dataset["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

#trainer.train(resume_from_checkpoint=True)
trainer.train()
print(tokenized_train_test_valid_dataset["test"])
test_input_data = tokenized_train_test_valid_dataset["test"]["input_ids"]

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.668812,0.621622
2,No log,0.697771,0.594595
3,No log,0.666479,0.648649
4,No log,0.695753,0.648649
5,No log,0.683936,0.621622
6,No log,0.680576,0.621622
7,No log,0.674869,0.621622
8,No log,0.671905,0.648649
9,No log,0.674588,0.621622
10,No log,0.674450,0.648649


Dataset({
    features: ['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 36
})


In [93]:
# predictions = trainer.predict(test_dataset=test_input_data)
# print(predictions)
trainer.evaluate(eval_dataset = tokenized_train_test_valid_dataset["test"])

{'eval_loss': 0.6006247997283936,
 'eval_accuracy': 0.7222222222222222,
 'eval_runtime': 0.3929,
 'eval_samples_per_second': 91.624,
 'eval_steps_per_second': 7.635,
 'epoch': 16.0}

In [94]:
trainer.predict(test_dataset = tokenized_train_test_valid_dataset["test"])

PredictionOutput(predictions=array([[-0.18069279, -0.3302801 ],
       [-0.9518123 ,  0.47074184],
       [-0.01721567, -0.4875356 ],
       [ 0.03432662, -0.54198587],
       [-0.16459821, -0.34577116],
       [-0.63464725,  0.15332495],
       [-0.48832142, -0.05141906],
       [-0.09577274, -0.40060315],
       [ 0.06358947, -0.5704366 ],
       [-0.00658918, -0.5122216 ],
       [-0.8930766 ,  0.4183537 ],
       [ 0.03362607, -0.5435797 ],
       [ 0.09024355, -0.6205659 ],
       [-0.53265965,  0.02194048],
       [-0.30711085, -0.22469023],
       [-0.0957941 , -0.38266465],
       [-0.16097395, -0.34766385],
       [ 0.09210249, -0.62115765],
       [-0.40707126, -0.10799819],
       [-0.1552012 , -0.35554382],
       [-0.5531868 ,  0.02562427],
       [-0.82378936,  0.34623808],
       [-0.52354175, -0.03060995],
       [ 0.39264107, -0.89842   ],
       [-0.67864734,  0.16242935],
       [-0.5714083 ,  0.04729527],
       [-0.73194873,  0.22674198],
       [ 0.374745  , -0.87

# Testing on IMDB Dataset

In [16]:
from datasets import load_dataset

imdb = load_dataset("imdb")

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)
tokenized_imdb = imdb.map(preprocess_function, batched=True)
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
import evaluate

accuracy = evaluate.load("accuracy")

import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [17]:
import sys
#sys.path.append("/home/sagemaker-user/bitsandbytes")
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import bitsandbytes

In [20]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

training_args = TrainingArguments(
    output_dir="text_classifier",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    evaluation_strategy="steps",  # Change evaluation strategy to "steps"
    eval_steps=100,  # Set the evaluation frequency to every 100 steps
    save_strategy="steps",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb['train'],
    eval_dataset=tokenized_imdb['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
100,No log,0.302497,0.876960
200,No log,0.250724,0.900440
300,No log,0.263815,0.904360


KeyboardInterrupt: 

In [6]:
conda list | grep cudatoolkit

cudatoolkit               11.8.0              h4ba93d1_12    conda-forge

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#11.8.0